# Task 3 - Feature Engineering

In [2]:
import numpy as np
from datasets import Dataset, ClassLabel, Sequence
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForTokenClassification
import pandas as pd
from sklearn.decomposition import TruncatedSVD
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.append(os.path.abspath("../src"))
import warnings
warnings.filterwarnings('ignore')

In [3]:
os.chdir("..")
print(os.getcwd()) 

d:\KAIM 5,6,7\Week-5\Credit-Risk-Probability-Model


In [4]:
from  train_model import *

In [5]:
import pandas as pd
# from src.processing.data_pipeline import prepare_data

df = pd.read_csv("Data/raw/data.csv")
df_clean, pipeline, final_features = prepare_data(df)

X = df_clean[final_features]

# Optional: transform for model input
try:
    X_transformed = pipeline.fit_transform(X)
    print("Transformed data shape:", X_transformed.shape)
except Exception as e:
    print(f" Transformation failed: {e}")

 Cleaning and transforming raw data...
 Preprocessing complete.
Transformed data shape: (95662, 11062)


In [6]:
svd = TruncatedSVD(n_components=20)
X_reduced = svd.fit_transform(X_transformed)
print("Reduced shape:", X_reduced.shape)
X_reduced 

Reduced shape: (95662, 20)


array([[ 2.02674976e+00, -1.18514959e-01,  6.39072719e-01, ...,
         1.23013373e-02, -7.72346538e-02,  2.73766361e-02],
       [ 2.60718123e+00, -1.69353557e-01,  7.69103213e-01, ...,
        -3.59725759e-02, -4.55830112e-04,  8.63441683e-03],
       [ 1.98301715e+00, -1.70486507e-01,  1.14221033e+00, ...,
        -3.05402736e-01,  8.44440530e-01, -7.13063791e-02],
       ...,
       [ 2.56718256e+00, -4.91679543e-02, -3.33972538e-03, ...,
        -3.85221133e-02, -1.61373021e-02, -6.27204335e-03],
       [ 1.73388624e+00, -3.82743919e-03, -1.91696326e-01, ...,
         1.31850397e-02, -1.88823953e-01, -2.23429797e-02],
       [ 2.56096250e+00, -7.88926625e-02, -2.29785303e-01, ...,
        -4.14038723e-02, -9.21471489e-03, -8.11915115e-03]],
      shape=(95662, 20))

In [7]:
# # # Usage:
# train_and_evaluate_models("Data/processed/cleaned_data.csv")